In [3]:
# !pip3 install -r requirement.txt


In [4]:

from PIL import Image
import os
from pathlib import PureWindowsPath, PurePosixPath,Path
import pandas as pd

In [1]:
!pwd

/home/ec2-user/SageMaker/segment-map


In [6]:
DATA_DIR = '/home/ec2-user/SageMaker/datasets/CIHP/img'

new = Image.new(mode="RGB", size=(256,256))
# new.save(DATA_DIR + '/gg.jpg')

def save_img(root1, label, new, file):
    label_src_root = root1.replace('img', label)
    if not os.path.exists(label_src_root):
        os.makedirs(label_src_root)
    label_src = os.path.join(label_src_root,file)
    new.save(label_src)

def create_empty_images():
    for (root,dirs,files) in os.walk(DATA_DIR, topdown=True):
        for file in files:
            # root1=str(PurePosixPath(PureWindowsPath(root)))
            # root1 =root1.replace('D:\/', 'D:/')
            # print(root)
            file = file.replace('jpg', 'png')
            save_img(root, 'labels',new, file)
            save_img(root, 'edges',new, file)
            save_img(root, 'labels_rev',new, file)

def path_replace(root1, label, file):
    label_src_root = root1.replace('img', label)
    return os.path.join(label_src_root,file)

def get_rel_path(root_path, idx_name, file): 
    src = path_replace(root_path, idx_name, file)  
    path = Path(src)
    index = path.parts.index(idx_name)
    return  "/".join(path.parts[index:])


def get_data():
    img_path_list,label_path_list, edge_path_list,f_name_list = [] ,[],[],[]
    out_path_list = []
    out_root = '/home/ec2-user/SageMaker/output/cihp_parsing_maps/'

    for (root,dirs,files) in os.walk(DATA_DIR, topdown=True):
        for file in files[:10]:
            # root1=str(PurePosixPath(PureWindowsPath(root)))
            # root1 =root1.replace('D:\/', 'D:/')

            img_root= get_rel_path(root, 'img', '')
            out_path = os.path.join(out_root,img_root)
            out_path_list.append(out_path)

            img_path= get_rel_path(root, 'img', file)
            img_path_list.append(img_path)

            file = file.replace('jpg', 'png')
            label_path= get_rel_path(root, 'labels', file)
            label_path_list.append(label_path)
            edge_path= get_rel_path(root, 'edges', file)
            edge_path_list.append(edge_path)
            f_name = file.split(".")[0]
            f_name_list.append(f_name)
    return img_path_list,label_path_list, edge_path_list, f_name_list,out_path_list

In [7]:
# create_empty_images()

In [8]:
img_path_list,label_path_list, edge_path_list,f_name_list,out_path_list = get_data()
df = pd.DataFrame({"img":img_path_list, "label": label_path_list, "edges": edge_path_list, "f_name": f_name_list, "out_root": out_path_list, })
val_path = '/home/ec2-user/SageMaker/datasets/CIHP/list/val.csv'
df.to_csv(val_path)